## Data file

In [5]:
import os
import olefile
import zlib
import struct

file_path = os.path.join(os.getcwd(), 'Data')
file_name = os.listdir(file_path)
file_name

['LEET_2023_L.hwp', 'PSAT_2023_L.hwp']

### HWP2PDF

In [6]:
import win32com.client as win32
import win32gui

os.chdir(file_path)

hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
hwnd = win32gui.FindWindow(None, "빈 문서 1 - 한글")

win32gui.ShowWindow(hwnd, 0)
hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

for i in file_name:
    hwp.Open(os.path.join(file_path, i))  # 한/글로 열어서
    hwp.HAction.GetDefault('FileSaveAsPdf', hwp.HParameterSet.HFileOpenSave.HSet)  # PDF로 저장할 건데, 설정값은 아래와 같이.
    hwp.HParameterSet.HFileOpenSave.filename = os.path.join(file_path, i.replace('.hwp', 'pdf'))  # 확장자는 .pdf로,
    hwp.HParameterSet.HFileOpenSave.Format = 'PDF'  # 포맷은 PDF로,
    hwp.HAction.Execute('FileSaveAsPdf', hwp.HParameterSet.HFileOpenSave.HSet) 
    
win32gui.ShowWindow(hwnd, 5)
hwp.XHwpDocuments.Close(isDirty=False)
hwp.Quit()

del hwp
del win32

ModuleNotFoundError: No module named 'win32com'

### PDF2Text

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader('./Data/PSAT_2023.pdf')

raw_txt = ''

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_txt += text

### HWP2Text

In [8]:
def get_hwp_text(file_name):
    hwp_text = []
    for file in file_name:
        if file.endswith('.hwp'):
            hwp = olefile.OleFileIO(os.path.join(file_path, file))
            hwp_text.append(hwp.openstream('PrvText').read().decode('UTF-16'))
    return hwp_text

hwp_text = get_hwp_text(file_name)
hwp_text

['< 15><2021학년도 법학적성시험><1>\r\n<<제 1 교시>><언어이해><<홀수형>>\r\n<><<성명><>    <수험번호><><><><><><><>>\r\n<>◦ 이 문제지는 30문항으로 구성되어 있습니다. 문항 수를 확인하십시오.\r\n◦ 문제지의 해당란에 성명과 수험번호를 정확히 쓰십시오.\r\n◦ 답안지에 수험번호, 문제유형, 성명, 답을 표기할 때에는 ‘답안 작성 시 반드시 지켜야 하는 사항’에 따라 표기하십시오.\r\n◦ 답안지의 ‘필적 확인란’에 해당 문구를 정자로 기재하십시오. \r\n\r\n\r\n[1~3] 다음 글을 읽고 물음에 답하시오.\r\n\r\n비즈니스 프로세스는 고객 가치 창출을 위해 기업 또는 조직에서 업무를 처리하는 과정을 말한다. 업무 처리 과정을 업무흐름도로 도식화하는 과정을 프로세스 모델링이라 하며, 그 결과물을 프로세스 모델이라고 한다. 프로세스 모델은 업무 처리 활동 및 활동들 간의 경로로 구성된다. 프로세스 모델이 효율적으로 작동하고 있는지를 확인, 분석, 수정․보완, 개선하는 작업이 필요한데, 프로세스 마이닝은 그중 한 기법이다. 프로세스 마이닝은, 시뮬레이션처럼 실제 이벤트 로그 수집 이전에 정립한 프로세스 모델 중심 분석기법과, 데이터 마이닝처럼 프로세스를 고려하지 않는 데이터 중심 분석기법을 연결하는 역할을 한다.\r\n프로세스 마이닝은 정보시스템을 통해 확보한 이벤트 로그에서 프로세스에 관련된 가치 있는 정보를 추출하는 것이다. 이벤트 로그란 정보시스템에 축적된 비즈니스 프로세스 수행 기록인데, 이것이 프로세스 마이닝의 출발점이 된다. 이벤트 로그는 행과 열로 표현되는 이차원 표 형태이다. 업무 활동으로 발생한 이벤트는 행으로 추가되며, 각 열에는 이벤트의 속성들이 기록된다. 이때 기록되는 속성으로 필수적인 것은 사례 ID, 활동명, 발생 시점이며, 다양한 분석을 위해 그 외 속성들도 추가될 수 있다. 이벤트 로그는 사용자에게 도움이 되는 정보를 직접 제공할 수 없는 원데이터이므로, 그것을 우리가 

## Dataset

In [ ]:
''' 
['<2009학년도 법학적성시험 예비시험 ><1>\r\n<<제 1 교시>><언어이해><<홀수형>>\r\n<  ><<성명><>    <수험 번호><0><1><－><><><><><>>\r\n<>◦ 이 문제지는 40문항으로 구성되어 있습니다. 문항 수를 확인하십시오.\r\n◦ 문제지와 답안지에 성명과 수험 번호를 정확하게 표기하십시오.\r\n◦ 답안지에 수험 번호와 답을 표기할 때에는 답안지 우측에 있는 ‘답안 작성시 반드시 지켜야 하는 사항’에 따라 표기하십시오.\r\n\r\n\r\n1. 관용 표현의 쓰임이 적절하지 않은 것은?\r\n① 요즘 사업이 계속 게걸음만 치고 있다.\r\n② 대변인은 성명서의 요지를 알기 쉽게 변죽을 치며 설명했다.\r\n③ 김 과장은 돌아서면서도 불만스러운 듯 고추 먹은 소리를 했다.\r\n④ 눈에 넣어도 아프지 않던 아이가 벌써 결혼을 앞둔 늠름한 청년이 됐구나.\r\n⑤ 박 선생은 오지랖이 넓어 남들 걱정은 잘하는데, 정작 자기 앞가림을 못해요.\r\n  \r\n  \r\n\r\n2. <보기>에 제시된 문장의 중의성(重義性)을 [  ]의 뜻에 맞게 해소하는 방법으로 적절하지 않은 것은?\r\n<><<보 기>><>\r\n<><>\r\n<ㄱ. 김 사장은 밝은 표정으로 환영 나온 사람들과 일일이 악수를 나누었다. [김 사장의 표정이 밝음]  ㄴ. 철수는 이 선배와 지도 교수를 방문하여 진로 문제를 상담했다. [철수가 두 사람을 방문함] ㄷ. 그는 조교를 통해 김 교수의 책을 전달받았다. [저자가 김 교수인 책] ㄹ. 김동규와 박은재가 올해 결혼하였다. [둘이 부부가 되었음] ㅁ. 나는 어제 철수만 만나지 않았다. [철수를 만났음]>\r\n① ㄱ의 중의성은 꾸밈을 받는 말의 불명확성 때문에 발생한 것인데, ‘밝은 표정으로’를 ‘사람들과’의 뒤로 옮긴다.\r\n② ㄴ의 중의성은 목적어의 불명확성 때문에 발생한 것인데, 주어 뒤에 반점(,)을 첨가한다.\r\n③ ㄷ의 중의성은 조사의 중의성 때문에 발생한 것인데, ‘의’를 ‘가 지은’으로 바꾼다. \r\n④ ㄹ의 중의성은 조사의 결합 구조 때문에 발생한 것인데, ‘와’와 ‘가’를 서로 교체한다.\r\n⑤ ㅁ의 중의성은 부정']
'''

https://github.com/NomaDamas/KICE_slayer_AI_Korean

In [ ]:
'''
[
  {
    "id": "2019_11_KICE_1-3",
    "paragraph": " 다음은 라디오 방송이다. 물음에 답하시오.혹시 어두운 밤길을 걸어 본 적이 있으신가요? 예전에 제가 밤길을 혼자 걸은 적이 있는데요, 처음엔 어둡고 무서웠지만 달빛 덕분에 어렵지 않게 걸었답니다. 여러분의 삶에 든든한 달빛 같은 방송, 청취자의 사연을 읽고 상담해 주는 ‘나에게 말해 줘’ 시간입니다. 저는 이 방송의 진행자인 심리 상담가 ○○○입니다. 오늘의 사연을 읽어 드릴게요. <사연> 저는 고등학생 □□라고 해요. 제 친구는 자꾸 친구들과 비교하면서 자신이 못났다고 생각해요. 차분하고 손재주도 좋은 친구인데 스스로를 그렇게 생각하는 게 안타까워요. 또 작은 실수에도 '난 항상 이래.'라며 자책하고 우울해해요. 그런 생각을 안 하도록 돕고 싶은데 방법을 모르겠어요. □□ 님은 스스로를 못났다고 생각하는 친구를 돕고 싶은데 방법을 모르신다는 거네요. 친구를 생각하는 마음이 참 따뜻하게 느껴져요. 저도 □□ 님처럼 안타깝네요.자신의 능력과 가치에 대한 전반적인 평가와 태도를 나타내는 말을 자존감이라고 합니다. 자존감이 낮은 원인은 다양하지만 일반적으로 알려진 것에는 남과 비교하는 버릇이 원인인 경우와 자책하는 태도가 원인인 경우가 있습니다. 사연 속 친구는 자신을 다른 사람과 비교해서 열등감을 느끼고, 사소한 실수에도 자신을 탓하며 스트레스를 받아서 자존감이 낮아진 것으로 보이네요.이러한 경우에는 ‘장점 말해 주기’와 ‘감정 헤아려 주기’ 방법이 도움이 될 수 있어요. 먼저 친구가 현재 가지고 있는 긍정적인 면들을 자주 말해 주세요. 그러면 친구가 자신의 장점을 깨닫고 남과 비교하지 않을 거예요. 그리고 친구의 마음을 헤아려 주세요. 만약 친구가 실수해서 자책하고 있으면 “많이 속상하겠구나. 괜찮아. 누구나 그럴 수 있어.”라며 친구의 감정을 이해해 주는 식으로요. 그러면 친구가 스스로 괜찮다고 느껴 스트레스를 덜 받고 자책하지 않을 거예요.오늘 방송 잘 들으셨나요? 저에게 하고 싶은 말이나 청취 소감은 언제든 게시판에 올려 주세요. 그럼 △△의 노래 ‘우리 함께’를 들으며 오늘 방송 마치겠습니다. 추운 날씨에 감기 조심하세요.",
    "problems": [
      {
        "question": " 위 방송 진행자의 말하기 방식에 대한 설명으로 가장 적절한 것은?",
        "score": 2,
        "choices": [
          "사연 내용을 정리하고 사연 신청자의 마음에 공감하고 있다.",
          "사연 신청자의 궁금증을 해소하고 다음 방송을 예고하고 있다.",
          "사연 내용을 선정하게 된 동기를 밝히고 청취자의 참여를 독려하고 있다.",
          "사연과 관련된 자신의 과거 경력을 소개하고 전문성을 부각하고 있다.",
          "사연에 대한 상담 중에 질문을 던지고 사연 속 상황을 다양한 관점에서 생각해 보도록 유도하고 있다."
        ],
        "answer": 1
      },
      {
        "question": " 다음은 위 방송을 진행하기 위해 진행자가 세운 계획이다. 방송에 반영되지 않은 것은?",
        "question_plus": "[오프닝] 방송의 취지를 드러내기 위해 ‘달빛’ 이야기로 시작 [사연 소개 및 고민 진단]◦ 사연 신청자가 보낸 사연 소개◦ ㉠(내용의 이해를 돕기 위해 자존감이라는 용어의 의미 제시) ◦ ㉡(자존감이 낮은 원인 중 일반적으로 알려진 원인을 제시하고 사연의 문제 상황에 적용 ) ◦ ㉢(사연의 문제 상황을 설명하기 위해 유사한 문제 상황 제시) [방법 제시]◦ ㉣(‘장점 말해 주기’ 방법을 안내하고 효과 제시) ◦ ㉤(‘감정 헤아려 주기’ 방법을 예를 들어 소개하고 효과 제시) [클로징] 청취자 게시판에 관한 안내 및 인사말로 마무리",
        "score": 2,
        "choices": [
          "㉠",
          "㉡",
          "㉢",
          "㉣",
          "㉤"
        ],
        "answer": 3
      },
      {
        "question": " 다음은 위 방송을 들은 청취자들이 게시판에 올린 댓글이다. 방송 내용을 고려하여 청취자들의 반응을 분석한 것으로 적절하지 않은 것은? [3점]",
        "question_plus": "'나에게 말해 줘' 게시판 - ○월 ○일 방송에 대해 자유롭게 의견을 남겨 주세요. - 청취자 1 : 저도 자존감이 낮은 거 같아서 좋은 방법이 나오기를 기다리며 들었는데, 스스로 자존감을 높이는 방법은 안 나오네요. - 청취자 2 : 자존감을 높여 주려면 자기만 부족하다는 생각에서 벗어나게 해 주라는 거네요. 그렇다면 가능한 목표를 세워서 도달하게 하는 방법도 성취감을 느낄 수 있게 해 주어 자존감을 높이는 데 도움이 되겠군요. - 청취자 3 : 딸아이의 자존감이 향상되도록 앞으로는 제 아이에게 긍정적인 면들을 말해 줘야겠어요. - 청취자 4 : 도와주고 싶은 대상의 연령대가 사연 속 친구와 다를 때에도 방송에서 알려 준 방법대로 해도 되는 건가요? - 청취자 5 : 감정을 헤아려 주는 건 좋은 방법이네요. 제가 직설적으로 말하는 버릇이 있어서 친구들이 속상했을 텐데 활용해 볼께요.",
        "score": 3,
        "choices": [
          "‘청취자 1’은 자신이 방송을 들은 목적과 관련해 방송 내용이 충분하지 않다고 판단하고 있군.",
          "‘청취자 2’는 방송 내용을 이해한 바를 확인하고 방송에서 안내되지 않았던 방법의 효과를 예측하고 있군.",
          "‘청취자 3’은 방송에서 언급한 방법을 다른 사람들에게 권유하고 적용할 것을 다짐하고 있군.",
          "‘청취자 4’는 방송에서 제시한 방법을 다른 경우에도 적용할 수 있는지 궁금해하고 있군.",
          "‘청취자 5’는 방송에서 언급한 방법을 긍정적으로 평가하고 자신의 언어 습관을 반성하고 있군."
        ],
        "answer": 3
      }
    ]
  }
]
'''